# 인스타그램 - 제주맛집

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
import unicodedata, re
from tqdm.notebook import tqdm

In [6]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

In [10]:
with open('password.txt') as f:
    Password = f.read()

In [ ]:
email = 'higoodgood'
input_email = driver.find_element_by_css_selector('._2hvTZ pexuQ zyHYP')
input_email.clear()
input_email.send_keys(email)

In [ ]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)

In [ ]:
input_pwd.submit()
time.sleep(1)

In [ ]:
try:
    driver.find_element_by_css_selector('').click()
    time.sleep(1)
except:
    pass